<a href="https://colab.research.google.com/github/KunChen-tw/DeepLearning/blob/main/deeplearning1109_Ch7_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The comparison between RNN and LSTM in temperature prediction (1)


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
torch.manual_seed(18)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming there is a temperature dataset
# Load the data, e.g., data.csv contains date and temperature
data = pd.read_csv('temperature_data.csv')  # Ensure the file contains date and temperature columns
temperature_data = data['temperature'].values  # Extract temperature data


Using device: cuda


# The comparison between RNN and LSTM in temperature prediction (2)

In [3]:
# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
temperature_data = scaler.fit_transform(temperature_data.reshape(-1, 1)).flatten()

# Build sequence data
seq_length = 10  # Use the temperature of the previous 10 days to predict the next day
X = []
y = []
for i in range(len(temperature_data) - seq_length):
    X.append(temperature_data[i:i+seq_length])
    y.append(temperature_data[i+seq_length])

X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-3-320cbdf1f303>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1).to(device)


# Define the LSTM and RNN model


In [6]:
# Define LSTM model
class TemperatureLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1):
        super(TemperatureLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the last output of LSTM
        return out

# Define RNN model
class TemperatureRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1):
        super(TemperatureRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Take the last output of RNN
        return out

# Initialize models, loss function, and optimizer
lstm_model = TemperatureLSTM().to(device)
rnn_model = TemperatureRNN().to(device)
!pip install torchinfo
from torchinfo import summary
print(summary(lstm_model))
print(summary(rnn_model))

Layer (type:depth-idx)                   Param #
TemperatureLSTM                          --
├─LSTM: 1-1                              10,600
├─Linear: 1-2                            51
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
Layer (type:depth-idx)                   Param #
TemperatureRNN                           --
├─RNN: 1-1                               2,650
├─Linear: 1-2                            51
Total params: 2,701
Trainable params: 2,701
Non-trainable params: 0


# Training RNN and LSTM model

In [14]:
# Initialize models, loss function, and optimizer
criterion = nn.MSELoss()

# Training function
def train_model(model, X_train, y_train, optimizer, epochs=1000):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# Initialize optimizers
lstm_optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.05)
rnn_optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.05)

# Train LSTM model
print("Training LSTM model...")
train_model(lstm_model, X_train, y_train, lstm_optimizer)

# Train RNN model
print("Training RNN model...")
train_model(rnn_model, X_train, y_train, rnn_optimizer)


Training LSTM model...
Epoch 0, Loss: 0.0074
Epoch 10, Loss: 0.0104
Epoch 20, Loss: 0.0114
Epoch 30, Loss: 0.0090
Epoch 40, Loss: 0.0081
Epoch 50, Loss: 0.0077
Epoch 60, Loss: 0.0077
Epoch 70, Loss: 0.0077
Epoch 80, Loss: 0.0076
Epoch 90, Loss: 0.0076
Epoch 100, Loss: 0.0076
Epoch 110, Loss: 0.0076
Epoch 120, Loss: 0.0076
Epoch 130, Loss: 0.0076
Epoch 140, Loss: 0.0076
Epoch 150, Loss: 0.0076
Epoch 160, Loss: 0.0075
Epoch 170, Loss: 0.0075
Epoch 180, Loss: 0.0075
Epoch 190, Loss: 0.0075
Epoch 200, Loss: 0.0075
Epoch 210, Loss: 0.0075
Epoch 220, Loss: 0.0075
Epoch 230, Loss: 0.0075
Epoch 240, Loss: 0.0075
Epoch 250, Loss: 0.0075
Epoch 260, Loss: 0.0075
Epoch 270, Loss: 0.0075
Epoch 280, Loss: 0.0075
Epoch 290, Loss: 0.0075
Epoch 300, Loss: 0.0075
Epoch 310, Loss: 0.0075
Epoch 320, Loss: 0.0075
Epoch 330, Loss: 0.0075
Epoch 340, Loss: 0.0075
Epoch 350, Loss: 0.0075
Epoch 360, Loss: 0.0075
Epoch 370, Loss: 0.0075
Epoch 380, Loss: 0.0075
Epoch 390, Loss: 0.0075
Epoch 400, Loss: 0.0075
Epoc

# Test model

In [15]:
# Test model and calculate average absolute error over an interval
def test_model(model, X_test, y_test, scaler, interval=30):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for i in range(interval):
            test_seq = X_test[i].unsqueeze(0)
            prediction = model(test_seq)
            predicted_temp = scaler.inverse_transform(prediction.cpu().numpy().reshape(-1, 1)).flatten()[0]
            actual_temp = scaler.inverse_transform(y_test[i].cpu().numpy().reshape(-1, 1)).flatten()[0]
            predictions.append(predicted_temp)
            actuals.append(actual_temp)

    # Calculate mean absolute error (MAE) over the interval and display predictions vs. actual values
    mae = mean_absolute_error(actuals, predictions)
    print(f'Interval Mean Absolute Error (MAE): {mae:.4f}')

# Test LSTM model performance over an interval
print("Testing LSTM model over interval...")
test_model(lstm_model, X_test, y_test, scaler)

# Test RNN model performance over an interval
print("Testing RNN model over interval...")
test_model(rnn_model, X_test, y_test, scaler)



Testing LSTM model over interval...
Interval Mean Absolute Error (MAE): 1.5975
Testing RNN model over interval...
Interval Mean Absolute Error (MAE): 1.8236
